# Title Classification

En este documento se empezará a crear la base de datos que se utilizará para la vector store de nuestro RAG para el chatbot.

Teniendo una lista de +19500 ariculos relacionados con la ciencia del deporte, debemos atacar los que puedan ser relevantes para nuestro chatbot en un principio, evitando tener muchos documentos que puedan no servirnos para nuestro propósito.

In [1]:
import numpy as np
import pandas as pd
from transformers import pipeline
import os
workpath = os.getcwd()

In [ ]:
# Cargar el pipeline de clasificación zero-shot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Lista de posibles clases
candidate_labels = ["calisthenics", "excercise", "posture", "nutrition", "training", "posture", "massage", "recovery"]

# Sacar titulos
data = pd.read_csv('./Docs/EstudiosTotal.csv', sep=',').drop_duplicates(subset='Title')
titulos = data['Title'].to_list()
clase = []



In [ ]:
candidate_labels = ["calisthenics", "excercise", "posture", "nutrition",
                    "training", "posture", "massage", "recovery", "biomechanics"]
# Clasificar los títulos
for titulo_index in range(len(titulos)):
    resultado = classifier(titulos[titulo_index], candidate_labels)
    clase.append(resultado['labels'][0])

In [17]:
data['Tipo'] = clase[:15450]
data.to_csv('./Docs/EstudiosTotalClasificadoBien.csv', sep=',')

In [ ]:
data.loc[data['Tipo'].isin(['excercise', 'posture', 'calisthenics', 'training'])].to_csv('./Docs/EstudiosEntrenamiento.csv', sep=';')

## Automatizar descarga

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def load_urls(df_path):
    df = pd.read_csv(df_path, sep=',')
    urls = df.loc[df['Label']==1]['URL'].tolist()
    return [url.strip() for url in urls]

def fetch_pdf_link(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    pdf_link = None
    for link in soup.find_all('a'):
        if 'pdf' in link.get('href', '').lower():
            pdf_link = link['href']
            break
    try:
        return url+pdf_link
    except:
        return pdf_link
def download_pdf(pdf_url, output_dir='pdfs'):
    try:
        response = requests.get(pdf_url)
        pdf_name = pdf_url.split('/')[-1]
        with open(f'{output_di}/{pdf_name}', 'wb') as file:
            file.write(response.rcontent)
    except:
        print('No se pudo descargar')
# Load URLs from file
urls = load_urls(workpath+'/Docs/PruebaArticulos.csv')

# Fetch PDF links
pdf_links = [fetch_pdf_link(url) for url in urls]
print(pdf_links)
# Create output directory if it doesn't exist
os.makedirs('pdfs', exist_ok=True)

# Download PDFs
for pdf_link in pdf_links:
    if pdf_link is not None:
        download_pdf(pdf_link)

**Importante:** La descarga no se pudo automatizar, por lo que tras la clasificación de los distintos documentos en base a su título se verifican y descargan de manera manual aquellos que se consideren más relevantes. No habrá tanta cantidad de archivos pero al menos será información útil.